## dependencies

In [76]:
import matplotlib.pyplot as plt
from skimage import data
from skimage import filters
from skimage import exposure
from skimage import io
from skimage import color
from skimage import morphology
from skimage import measure
from scipy import ndimage

import numpy as np
import cv2
import ntpath

from os import listdir
from os.path import isfile, isdir, join

## constants & onetimers

In [77]:
EROSION_SIZE = 15
DILATION_SIZE = 15

In [78]:
sift = cv2.xfeatures2d.SIFT_create()
bruteForceMatcher = cv2.BFMatcher()

## Classes

In [79]:
class Frame:
    def __init__(self, filePath):
        self.filePath = filePath
        self.fileName = ntpath.basename(filePath)
        self.imageSkimage = io.imread(filePath)
        self.grey = color.rgb2grey(self.imageSkimage)
        self.imageCV = cv2.imread(filePath)
        
        finger_mask, screen_mask = self.computeMasks()
        self.maskFinger = finger_mask
        self.maskScreen = screen_mask
        
        kp, desc = self.computeSIFTDescriptors()
        self.kp = kp
        self.desc = desc
        
        self.assignedScene = None
        
    def computeMasks(self):
        #locates area usable for image comparison
        
        # Otsu
        otsu_tresh = filters.threshold_otsu(self.grey)
        mask = (self.grey > otsu_tresh)

        # Erosion & dilation
        mask = morphology.binary_dilation(mask, morphology.square(DILATION_SIZE)).astype(int)
        mask = morphology.binary_erosion(mask, morphology.square(EROSION_SIZE))

        #For testing
        #mask = np.array([[1, 1, 1, 0],
        #                [1, 0, 0, 0],
        #                 [0, 0, 1, 0],
        #                 [0, 0, 1, 0],
        #                 [0, 0, 0, 0]])

        # Select largest component (screen)
        mask = measure.label(mask)
        regions = measure.regionprops(mask)
        largest_region_props = max(regions, key=lambda r: r.area)
        mask = (mask == largest_region_props.label).astype(int)
        mask = ndimage.binary_fill_holes(mask).astype(int)

        # Get final masks (ignore fingers as much as possible)
        finger_mask = mask
        screen_mask = (morphology.convex_hull_image(mask)).astype(int)
        return finger_mask, screen_mask
    
    def computeSIFTDescriptors(self):
        gray = cv2.cvtColor(self.imageCV, cv2.COLOR_BGR2GRAY)
        kp, desc = sift.detectAndCompute(gray, mask=self.maskFinger.astype(np.uint8))
        return kp, desc
    
    def displayMasks(self):
        plt.figure(figsize=(9, 4))
        plt.subplot(131)
        plt.imshow(self.imageSkimage, cmap='gray', interpolation='nearest')
        plt.axis('off')
        plt.subplot(132)
        plt.imshow(self.maskFinger, cmap='gray', interpolation='nearest')
        plt.axis('off')
        plt.subplot(133)
        plt.imshow(self.maskScreen, cmap='gray', interpolation='nearest')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

        
class EmptyFrame(Frame):
    def __init__(self):
        self.fileName = 'none'
        
emptyFrame = EmptyFrame()

## routines

In [80]:
def similaritySIFT(frame, scene):
    matches = bruteForceMatcher.knnMatch(frame.desc, scene.desc, k=2)
    total = 0
    good = 0
    for m,n in matches:
        total +=1
        if m.distance < 0.75*n.distance:
            good += 1 
    return good/total

In [81]:
def similaritySSIM(frame, scene):
    maskedFrame = frame.grey * frame.maskFinger
    maskedScene = scene.grey * frame.maskFinger #yes, frame's mask must be used
    return measure.compare_ssim(maskedFrame, maskedScene, None)

# Main experiment

In [90]:
# SELECT EXPERIMENT SAMPLE
sample_path = 'experiment-samples/extra-small'
#sample_path = 'experiment-samples/trivial'


# LOAD AND PREPROCESS SCENES (defined by the experiment analyst)

scenes_path = sample_path + '/scenes'
scene_files = [(scenes_path+'/'+f) for f in listdir(scenes_path) if isfile(join(scenes_path, f))]
scenes = []
for path in scene_files:
    frame = Frame(path)
    scenes.append(frame)

# FOREACH RECORDING
    # FOREACH FRAME
        #PREPROCESS
        #FOREACH SCENE
            #COMPUTE SIMILARITY TO SCENES
        #DETERMINE SCENE FOR FRAME
    # FILTER ORPHANS
    
recordings_path = sample_path + '/recordings'
recordings = [d for d in listdir(recordings_path) if isdir(join(recordings_path, d))]
for recording in recordings:
    recording_path = recordings_path + '/' + recording
    frame_files = [(recording_path+'/'+f) for f in listdir(recording_path) if isfile(join(recording_path, f))]
    frames = []
    for path in frame_files:
        frame = Frame(path) # Preprocessing takes place here
        print(frame.fileName)
        frames.append(frame)
        
        #FRAME GROUPING (TODO)
        
        bestSum = 0
        bestScene = emptyFrame
        for scene in scenes:
            simSIFT = similaritySIFT(frame, scene)
            simSSIM = similaritySSIM(frame, scene)
            if(bestSum < simSIFT + simSSIM and (1.15 < simSIFT + simSSIM or 0.45 < simSIFT)):
                bestSum = simSIFT + simSSIM
                bestScene = scene 
            print ("SIFT: " + str(simSIFT))
            print ("SSIM: " + str(simSSIM))
        frame.assignedScene = bestScene
        print ('     '+frame.assignedScene.fileName)
        print ()

        
    # FILTER ORPHANS 
    for frameNumber, currentFrame in enumerate(frames):
        if frameNumber == 0 : continue
        if frameNumber == len(frames)-1: continue
        previousScene = frames[frameNumber-1].assignedScene
        currentScene  = frames[frameNumber  ].assignedScene # (currentFrame)
        nextScene     = frames[frameNumber+1].assignedScene
        # Rule 1:
        if (currentScene is not previousScene) and (previousScene is nextScene):
            currentFrame.assignedScene = previousScene
            print('Rule 1 applied on frame '+str(frameNumber))
            continue
        # Rule 2:
        if (currentScene is not previousScene) and (currentScene is not nextScene) and (currentScene is not emptyFrame):
            currentFrame.assignedScene = emptyFrame
            print('Rule 2 applied on frame '+str(frameNumber))
            continue
    
# EVALUATE EXPERIMENT (TODO)
    


output_00136-1-intro.png
SIFT: 0.6301969365426696
SSIM: 0.986008185868
SIFT: 0.19912472647702406
SSIM: 0.796452759047
     scene-intro.png

output_00137-1-intro.png
SIFT: 0.5947242206235012
SSIM: 0.982149631355
SIFT: 0.1510791366906475
SSIM: 0.833753404753
     scene-intro.png

output_00137b-0-none.png
SIFT: 0.10480349344978165
SSIM: 0.791037127458
SIFT: 0.14847161572052403
SSIM: 0.907288303677
     none

output_00138-1-intro.png
SIFT: 0.5321100917431193
SSIM: 0.99091478422
SIFT: 0.14984709480122324
SSIM: 0.91253361977
     scene-intro.png

output_00139-1-intro.png
SIFT: 0.520618556701031
SSIM: 0.992529879137
SIFT: 0.15463917525773196
SSIM: 0.938306747565
     scene-intro.png

output_00140-1-intro.png
SIFT: 0.4325581395348837
SSIM: 0.964556850807
SIFT: 0.13023255813953488
SSIM: 0.89678023506
     scene-intro.png

output_00704-0-none.png
SIFT: 0.10480349344978165
SSIM: 0.791037127458
SIFT: 0.14847161572052403
SSIM: 0.907288303677
     none

output_00736-2-login.png
SIFT: 0.2487562189054